In [21]:
import re
import itertools
import pandas as pd
import nltk
import string
from nltk.tokenize import RegexpTokenizer
import pymorphy2
from nltk.stem.snowball import SnowballStemmer

In [2]:
with open('D:\\nltk-dost\data\\avidreaders.ru__prestuplenie-i-nakazanie-dr-izd.txt', 'r', encoding="utf-8") as file:
    data = file.read().rstrip()

In [4]:
def _parse_parts(text):
    return re.split(r"(Часть \\*[\w]+\\*\n)", text)

In [5]:
def _parse_chapters(text, part):
    return [
            {
                "part": part,
                "chapter_id": id_,
                "text": chapter_text
            }
                for id_, chapter_text in enumerate(re.split(r"(Глава [IVXLCDM])", text))
        ]

In [6]:
def clust_text(data):
    result = []
    parts = _parse_parts(data)[1:][1::2]
    for part_id, content in enumerate(parts):
        parts_content = _parse_chapters(content, part_id)[1:][1::2]
        for p_content in parts_content:
            p_content["chapter_id"] = int(p_content["chapter_id"]/2)
            p_content["text"] = p_content["text"].replace("\n", ' ')
        result.append(parts_content)
    return list(itertools.chain(*result))

In [7]:
clust_text_list = clust_text(data)

In [8]:
df_text = pd.DataFrame(clust_text_list)

In [ ]:
df_text.head()

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(nltk.corpus.stopwords.words('russian'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Marat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Marat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
morph = pymorphy2.MorphAnalyzer()
stemmer = SnowballStemmer("russian")
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(nltk.corpus.stopwords.words('russian'))

In [4]:
def clear_tokenize_text(text):
    tokens = tokenizer.tokenize(text)
    return [word.lower() for word in tokens if word not in string.punctuation and not word.isspace() and word.lower() not in stop_words and word.isalpha()]
    

In [19]:
def lemmatize_stemmer(tokens, lemm=True, stem=False):
    lemmas = tokens
    if lemm:
        lemmas = [morph.parse(word)[0].normal_form for word in tokens]
    if stem:
        return [stemmer.stem(word) for word in lemmas]
    return lemmas

In [14]:
df_text["text_clean_tokenized"] = df_text["text"].apply(lambda x: clear_tokenize_text(x)) 

In [17]:
df_text["text_lemm_stem"] = df_text["text_clean_tokenized"].apply(lambda x: lemmatize_stemmer(x, lemm=True, stem=True))
df_text["text_stem"] = df_text["text_clean_tokenized"].apply(lambda x: lemmatize_stemmer(x, lemm=False, stem=True))
df_text["text_lemm"] = df_text["text_clean_tokenized"].apply(lambda x: lemmatize_stemmer(x, lemm=True, stem=False))

In [21]:
with open("../data/parsed_text.json", "w", encoding="utf-8") as file:
    df_text.to_json(file, force_ascii=False)

In [69]:
abs_freq = pd.DataFrame(list(itertools.chain(*df_text["text_lemm_stem"].apply(list)))).value_counts()
freq = abs_freq / abs_freq.sum()